In [1]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl

Processing /kaggle/input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import datatable as dt
import pandas as pd
import numpy as np
from datatable import (dt, sd, f, by, ifelse, update, sort, count, min, max, mean, sum, rowsum)
import riiideducation
import matplotlib.pyplot as plt

In [3]:
TARGET = 'answered_correctly'

In [4]:
# cumsum
# 質問
# 正答数
# パート別質問
# パート別正答数
question_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')[['question_id','part']]
question_df['part'] = question_df['part'] - 1
question_df['part'] = question_df['part'].astype(np.int8)
cumsum_df = pd.read_pickle('../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip')[['row_id','user_id','content_type_id','content_id','answered_correctly']]
cumsum_df['answered_correctly'] = cumsum_df['answered_correctly'].astype(np.int8)
cumsum_df = cumsum_df.loc[cumsum_df['content_type_id'] == 0]
del cumsum_df['content_type_id']
cumsum_df = cumsum_df.set_index('content_id')
cumsum_df = cumsum_df.merge(question_df,how='left',left_index=True, right_on = 'question_id')

cumsum_df['u_cumsum'] = 1
cumsum_df['u_cumsum'] = cumsum_df.groupby('user_id')['u_cumsum'].cumsum().astype(np.int16)
cumsum_df['u_target_cumsum'] = 1
cumsum_df['u_target_cumsum'] = cumsum_df.groupby(['user_id','answered_correctly'])['u_target_cumsum'].cumsum().astype(np.int16)
cumsum_df.loc[cumsum_df['answered_correctly']==0, 'u_target_cumsum'] = 0


cumsum_df['up_cumsum'] = 1
cumsum_df['up_cumsum'] = cumsum_df.groupby(['user_id','part'])['up_cumsum'].cumsum().astype(np.int16)
cumsum_df['up_target_cumsum'] = 1
cumsum_df['up_target_cumsum'] = cumsum_df.groupby(['user_id','answered_correctly','part'])['up_target_cumsum'].cumsum().astype(np.int16)
cumsum_df['uq_cumsum'] = 1
cumsum_df['uq_cumsum'] = cumsum_df.groupby(['user_id','question_id'])['uq_cumsum'].cumsum().astype(np.int16)
cumsum_df['u_cumsum_mean'] = cumsum_df['u_target_cumsum'] / cumsum_df['u_cumsum']
cumsum_df['up_cumsum_mean'] = cumsum_df['up_target_cumsum'] / cumsum_df['up_cumsum']
cumsum_df[['row_id','u_cumsum','u_target_cumsum','up_cumsum','up_target_cumsum','uq_cumsum']].to_csv('cumsum.csv')
del cumsum_df, question_df

In [5]:
#cumsum_df['continuous_target'] = cumsum_df['user_id'].astype(str) + cumsum_df['answered_correctly'].astype(str)
#y = cumsum_df['continuous_target']
#cumsum_df['new'] = y.groupby((y != y.shift()).cumsum()).cumcount() + 1

In [6]:
train = dt.fread("../input/riiid-train-data-multiple-formats/riiid_train.jay")
questions = dt.fread("../input/riiid-test-answer-prediction/questions.csv")
lectures_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

# Preprocess

In [7]:
questions['part'] = f.part - 1
questions.names = {'question_id':'content_id'}
questions.key = "content_id"
train['timestamp'] = f.timestamp / 1000
train['prior_question_elapsed_time'] = f.prior_question_elapsed_time / 1000
#del train['row_id']

In [8]:
lectures_df['type_of'] = lectures_df['type_of'].replace('solving question', 'solving_question')
lectures_df = pd.get_dummies(lectures_df, columns=['part', 'type_of'])
part_lectures_columns = [column for column in lectures_df.columns if column.startswith('part')]
types_of_lectures_columns = [column for column in lectures_df.columns if column.startswith('type_of_')]
lectures_df.to_csv('tmp_lectures_df.csv',index = 'lecture_id')

# StatQuestionData

In [9]:
train = train[dt.f.content_type_id == 0, :]
del train['content_type_id']
train = train[:, :, dt.join(questions)]

In [10]:
# shift feature 
wrk = train[0, :, by('user_id','timestamp')]
wrk[:, update(shift_timestamp = dt.shift(f.timestamp),n=1), by("user_id")]
wrk = wrk[:, :, sort(-f.user_id, -f.timestamp)]
wrk[:, update(prior_question_elapsed_time = dt.shift(f.prior_question_elapsed_time),n=1), by("user_id")]
wrk[:, update(prior_question_had_explanation = dt.shift(f.prior_question_had_explanation),n=1), by("user_id")]
wrk[:, update(content_id = dt.shift(f.content_id),n=1), by("user_id")]
wrk = wrk[:, :, sort(f.user_id, f.timestamp)]
wrk['interval'] = f.timestamp - f.shift_timestamp
wrk.names = {'content_id':'prior_content_id'}
wrk = wrk[:,['user_id','timestamp','interval','prior_question_elapsed_time','prior_question_had_explanation','prior_content_id']]
wrk.key= (["user_id","timestamp"])
del train['prior_question_elapsed_time'],train['prior_question_had_explanation']
train = train[:,:,dt.join(wrk)]

In [11]:
content_count = train[:,['content_id','prior_content_id']][:, {'content_count': count(f.prior_content_id)},('content_id')]
content_count.key= (["content_id"])
prior_content_count = train[:,['content_id','prior_content_id','user_id']][:, {'prior_count': count(f.user_id)},by('content_id','prior_content_id')]
prior_content_count = prior_content_count[:,:,dt.join(content_count)]
prior_content_count['prior_position_per'] = f.prior_count / f.content_count
prior_content_count = prior_content_count[dt.f.prior_position_per > 0.001, :]
prior_content_count[:,['content_id','prior_content_id','prior_position_per']].to_csv('prior_position_per.csv')
del content_count,prior_content_count

# StatUserData

In [12]:
###  質問統計値
# 回答平均時間
# 回答時間のばらつき
# 正答率
# 回答数
# 間隔平均
# 回答ユーザー数
# リピート率
question_wrk = train[:, {'q_elapsed_time_mean': mean(f.prior_question_elapsed_time),
                         'q_elapsed_time_std': sd(f.prior_question_elapsed_time),
                         'q_correct_mean':mean(f.answered_correctly),
                         'q_count': count(f.answered_correctly),
                         'q_task_id_mean': mean(f.task_container_id),
                         'q_had_explanation_mean':mean(f.prior_question_had_explanation)},('content_id')]
question_wrk.key = (['content_id'])
questions = questions[:,:,dt.join(question_wrk)]
question_wrk2 = train[:, {'u_numquie': count(f.answered_correctly)}, by('user_id','content_id')][:, {'u_numquie': count(f.user_id)}, by('content_id')]
question_wrk2.key = (['content_id'])
questions = questions[:,:,dt.join(question_wrk2)]
questions['q_repeat_per'] = (f.q_count - f.u_numquie) / f.q_count
questions.head()

,content_id,bundle_id,correct_answer,part,tags,q_elapsed_time_mean,q_elapsed_time_std,q_correct_mean,q_count,q_task_id_mean,q_had_explanation_mean,u_numquie,q_repeat_per
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,0,0,51 131 162 38,19.6937,8.43723,0.907721,6903,757.922,0.952844,6380,0.0757642
1,1,1,1,0,131 36 81,19.3207,10.207,0.890646,7398,791.92,0.981045,6829,0.0769127
2,2,2,0,0,131 101 162 92,24.5748,10.3662,0.554281,44905,369.118,0.902114,38322,0.146598
3,3,3,0,0,131 149 162 29,21.629,9.29408,0.779437,22973,529.397,0.969317,20643,0.101423
4,4,4,3,0,131 5 162 38,22.0469,9.69508,0.613215,31736,378.574,0.533054,29006,0.0860222
5,5,5,2,0,131 149 162 81,22.1366,7.59776,0.861828,9727,725.362,0.979207,8957,0.0791611
6,6,6,2,0,10 94 162 92,23.9968,10.509,0.474545,56707,334.374,0.716599,47199,0.167669
7,7,7,0,0,61 110 162 29,21.8871,8.73641,0.866024,16585,458.558,0.986109,15167,0.0854989
8,8,8,3,0,131 13 162 92,22.8168,10.6995,0.90662,8535,660.708,0.968765,7895,0.0749854


In [13]:
questions.to_csv('question_stats.csv')

In [14]:
train[dt.f.answered_correctly == 1, :][:, {'correct_q_elapsed_time_mean': mean(f.prior_question_elapsed_time)},('content_id')].to_csv('correct_q_elapsed_time_mean.csv')

In [15]:
train[dt.f.answered_correctly == 0, :][:, {'incorrect_q_elapsed_time_mean': mean(f.prior_question_elapsed_time)},('content_id')].to_csv('incorrect_q_elapsed_time_mean.csv')

In [16]:
###  ユーザ統計値
# 回答平均時間に対する回答時間の平均
# 回答時間のばらつき
# 正答率
# 回答数
# 間隔平均
# リピート率
train = train[:,:,dt.join(question_wrk)]
train['u_elapsed_time_per'] = f.prior_question_elapsed_time / f.q_elapsed_time_mean

user_part = train[:, {'up_elapsed_time_per_mean': mean(f.u_elapsed_time_per),
                  'up_elapsed_time_per_std': sd(f.u_elapsed_time_per),
                  'up_correct_mean': mean(f.answered_correctly),
                  'up_count': count(f.answered_correctly),
                  'up_interval_mean': mean(f.interval)}, by('user_id','part')]

user_wrk = train[:, {'up_q_numquie': count(f.answered_correctly)}, by('user_id','content_id','part')][:, {'up_numquie': count(f.content_id)}, by('user_id','part')]
user_wrk.key = (['user_id','part'])
user_part = user_part[:,:,dt.join(user_wrk)]
user_part['up_repeat_per'] = (f.up_count - f.up_numquie) / f.up_count
user_part.head(3)

,user_id,part,up_elapsed_time_per_mean,up_elapsed_time_per_std,up_correct_mean,up_count,up_interval_mean,up_numquie,up_repeat_per
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,115,0,0.91112,0.157292,0.702703,37,38.1444,37,0
1,115,1,0.938027,NA,1,1,666393,1,0
2,115,2,0.560317,0,0.666667,3,110.132,3,0


In [17]:
user_part.to_csv("user_part_stats.csv")

In [18]:
users = train[:, {'u_elapsed_time_per_mean': mean(f.u_elapsed_time_per),
                  'u_elapsed_time_per_std': sd(f.u_elapsed_time_per),
                  'u_correct_mean': mean(f.answered_correctly),
                  'u_count': count(f.answered_correctly),
                  'u_interval_mean': mean(f.interval)}, by('user_id')]

user_wrk = train[:, {'u_q_numquie': count(f.answered_correctly)}, by('user_id','content_id')][:, {'u_numquie': count(f.content_id)}, by('user_id')]
user_wrk.key = (['user_id'])
users = users[:,:,dt.join(user_wrk)]
users['u_repeat_per'] = (f.u_count - f.u_numquie) / f.u_count
users.head()

,user_id,u_elapsed_time_per_mean,u_elapsed_time_per_std,u_correct_mean,u_count,u_interval_mean,u_numquie,u_repeat_per
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,115,0.973875,0.454233,0.695652,46,14856.6,46,0
1,124,0.889092,0.397394,0.233333,30,45.2647,30,0
2,2746,0.984537,0.153215,0.578947,19,46.4143,17,0.105263
3,5382,1.5614,1.02933,0.672,125,16948,120,0.04
4,8623,0.955695,0.373434,0.642202,109,14301.1,104,0.0458716
5,8701,1.00254,0.283455,0.588235,17,98.2057,17,0
6,12741,1.12462,0.294201,0.573585,265,62281.3,250,0.0566038
7,13134,1.01427,0.346506,0.706356,1243,16274.3,1223,0.0160901
8,24418,0.87804,0.28739,0.690275,6283,2744.28,5200,0.17237


In [19]:
users.to_csv("user_stats.csv")

# StatPartData

In [20]:
train[:,{'p_correct_mean': mean(f.answered_correctly),
         'p_elapsed_time_mean': mean(f.prior_question_elapsed_time),
         'p_explanation_mean': mean(f.prior_question_had_explanation),
         'p_task_id_mean' : mean(f.task_container_id),
         'p_count': count(f.answered_correctly)},by('part')].to_csv('part_stats.csv')

In [21]:
cumsum_df = dt.fread("./cumsum.csv")
del cumsum_df['C0']
cumsum_df.key = (['row_id'])
train = train[:,:,dt.join(cumsum_df)]
users.key =(['user_id'])
train = train[:,:,dt.join(users)]
questions.key = (['content_id'])
train = train[:,:,dt.join(questions)]
train.head()

,row_id,timestamp,user_id,content_id,task_container_id,user_answer,answered_correctly,bundle_id,correct_answer,part,…,q_count.0,q_task_id_mean.0,q_had_explanation_mean.0,u_numquie.0,q_repeat_per
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,115,5692,1,3,1,5692,3,4,…,36514,309.146,0.682555,33791,0.0745741
1,1,56.943,115,5716,2,2,1,5716,2,4,…,23991,435.177,0.523678,22547,0.0601892
2,2,118.363,115,128,0,0,1,128,0,0,…,19619,200.267,0.200775,19260,0.0182986
3,3,131.167,115,7860,3,0,1,7860,0,0,…,21733,196.748,0.281385,21261,0.0217181
4,4,137.965,115,7922,4,1,1,7922,1,0,…,19409,207.366,0.201041,19065,0.0177237
5,5,157.063,115,156,5,2,1,156,2,0,…,22493,219.441,0.310164,21908,0.0260081
6,6,176.092,115,51,6,0,1,51,0,0,…,24107,238.397,0.358154,23373,0.0304476
7,7,194.19,115,50,7,3,1,50,3,0,…,21401,226.072,0.276043,20856,0.0254661
8,8,212.463,115,7896,8,2,1,7896,2,0,…,22086,226.58,0.304803,21560,0.023816
